# Input Data Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/My Drive/amazing_grace/

/content/drive/.shortcut-targets-by-id/108Ujtp8i29zJ2_EUinbLFEbkDJZjf7jb/amazing_grace


In [ ]:
pip install librosa

In [ ]:
import tensorflow as tf
import librosa
import numpy as np
import os

Here we have our parse function, which essentially reads in our audio files into numpy arrays using the librosa library, and then adds them into array audio_examples. We also have an array with the labels where everything has "grace" for now.

In [ ]:
sh = (64)
audio_examples = []
audio_labels = []
count = 0
for audio_file in os.listdir('./'):
  if count == 1301:
    break
  curr_audio = librosa.load(audio_file)[0]
  curr_audio = np.resize(curr_audio,sh)
  #print(np.shape(curr_audio)) -> confirmation that resizing work, commented out because of large output
  curr_audio = curr_audio.tolist()
  audio_examples.append(curr_audio)
  audio_labels.append('grace') #if we have unique labels later then we'd have to load those in a separate loop
  count += 1

We stopped at 1301 files for testing purposes above which we can see below.

In [ ]:

print(len(audio_examples))
print(len(audio_labels))


1301
1301


Now, we want to convert our arrays into tensors by using tf.constant. Tutorials online replace this need by just loading in our files directly into the dataset, then using mapping and decode_wav, but we can't do this because our data is not in .wav format, which is why we had to first do the librosa functions above, then create our own tensors to load manually.

In [ ]:
audio_examples = tf.constant(audio_examples)
audio_labels = tf.constant(audio_labels)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((audio_examples, audio_labels))

Verify our dataset has expected contents:

In [ ]:
j = 0
for i in train_dataset:
  #if j == 1:
  #  break
  print(i)
  j +=1

Streaming output truncated to the last 5000 lines.
       -2.1362305e-04, -6.1035156e-05, -3.9672852e-04, -4.8828125e-04,
       -2.4414062e-04,  3.0517578e-05,  3.0517578e-05, -1.2207031e-04,
        0.0000000e+00,  1.2207031e-04,  3.3569336e-04,  2.1362305e-04,
       -1.5258789e-04, -1.5258789e-04, -2.7465820e-04, -1.5258789e-04],
      dtype=float32)>, <tf.Tensor: shape=(), dtype=string, numpy=b'grace'>)
(<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ 1.8310547e-04,  3.9672852e-04,  3.0517578e-05,  3.0517578e-04,
        6.1035156e-05, -9.1552734e-05, -1.5258789e-04,  9.1552734e-05,
        6.1035156e-05, -2.1362305e-04, -3.0517578e-05,  9.1552734e-05,
       -3.0517578e-05, -6.1035156e-05,  6.1035156e-05, -6.1035156e-05,
       -9.1552734e-05,  2.4414062e-04,  3.3569336e-04,  0.0000000e+00,
        0.0000000e+00, -2.1362305e-04, -5.1879883e-04, -3.0517578e-04,
       -2.4414062e-04, -2.1362305e-04, -2.4414062e-04, -9.1552734e-05,
       -2.1362305e-04, -6.1035156e-05,  0.0

Shuffle our dataset's contents to prepare for batching.

In [ ]:
train_dataset = train_dataset.shuffle(len(audio_examples))

Here, you can specify size you want per batch, we have indicated 1000 for now.

In [ ]:
batch1 = train_dataset.batch(1000) # num samples we want per batch

In [ ]:
list(batch1.as_numpy_iterator())

[(array([[ 1.4648438e-03,  2.6245117e-03,  3.0822754e-03, ...,
          -3.7231445e-03, -3.7841797e-03, -2.3803711e-03],
         [ 8.8500977e-04,  7.6293945e-04, -3.0517578e-04, ...,
          -1.2512207e-03, -1.7089844e-03, -2.1362305e-03],
         [-3.0517578e-05, -1.2207031e-04, -3.9672852e-04, ...,
          -7.9345703e-04, -2.1362305e-04,  1.2207031e-04],
         ...,
         [-6.1035156e-05, -1.5258789e-04, -1.2207031e-04, ...,
           3.0517578e-04,  1.5258789e-04,  0.0000000e+00],
         [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [-2.4414062e-04, -9.1552734e-05,  1.3427734e-03, ...,
          -6.7138672e-04,  1.8310547e-04, -7.6293945e-04]], dtype=float32),
  array([b'grace', b'grace', b'grace', b'grace', b'grace', b'grace',
         b'grace', b'grace', b'grace', b'grace', b'grace', b'grace',
         b'grace', b'grace', b'grace', b'grace', b'grace', b'grace',
         b'grace', b'grace', b

For fu you can add in parallelization using prefetch() and other tools.